<a href="https://colab.research.google.com/github/chanhyeong00/machine_learning_study/blob/main/pytorch/%EC%8B%AC%ED%99%941(%EA%B3%BC%EB%8C%80%2C%20%EA%B3%BC%EC%86%8C%EC%A0%81%ED%95%A9%20%EB%AC%B8%EC%A0%9C%20%ED%95%B4%EA%B2%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 배치 정규화
평균은 0.0, 분산은 1.0으로 정규화된다. 수식은 p.167

2D/3D 입력 데이터

In [1]:
# 기본값
import torch
from torch import nn


x = torch.FloatTensor(
    [
        [-0.6577, -0.5797, 0.6360],
        [0.7392, 0.2145, 1.523],
        [0.2432, 0.5662, 0.322]
    ]
)
print(nn.BatchNorm1d(3)(x)) # 3 은 num_features

tensor([[-1.3246, -1.3492, -0.3756],
        [ 1.0912,  0.3077,  1.3685],
        [ 0.2334,  1.0415, -0.9930]], grad_fn=<NativeBatchNormBackward0>)


채널이 포함된 입력 데이터를 정규화한다면 BatchNorm2d 클래스를 사용한다.(4d 입력 데이터)

이 외에도 여러가지 정규화가 있다. p.169 참고

## 가중치 초기화(weight Initialization)

모델 매개변수에 적절한 초깃값을 설정한다면 기울기 폭주나 기울기 소실 문제를 완화할 수 있다

### 상수 초기화
가중치를 초기화하는 매우 간단한 방법은 상숫값으로 초기화하는 것. 상수 초기화는 초기 가중치 값을 모두 같은 값으로 초기화한다.

- 0, 1, 특정값, 단위행렬(Unit metrix), 디렉델타함수(Dirac Delta Function) 값 등이 있다.

일반적으로 거의 사용되지 않는다. 모든 가중치 초기값을 같은 값으로 초기화하면 배열 구조의 가중치에서는 문제가 발생한다.

이러한 문제는 **대칭파괴(Breaking Symmetry) 현상으로 인해 모델 학습이 어렵거나 불가능하게 만든다.(모든 노드 동일한 출력 생성하여 모델이 학습되지 않음)

가중치값이 모두 0으로 초기화된 경우, 역전파 과정에서 모든 가중치가 동일한 값으로 갱신된다. 이 경우 역전파를 통해 가중치가 제대로 갱신되지 않으므로 학습이 정상적으로 진행되지 않는다.


상수 초기화는 스칼라값을 입력으로 받는 매우 작은 모델이나 퍼셉트론 등에 적용하거나 편향(bias)을 초기화하는 경우 0이나 0.01 등의 형태로 초기화하는 데 사용

### 무작위 초기화
초기 가중치를 무작위 값이나 특정 분포 형태로 초기화하는 것을 말한다. 예를 들어 정규분포의 형태로 가중치의 값을 초기화한다.

여러 방법은 p.171--

In [ ]:
from torch import nn

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.layer = nn.Sequential(
        nn.Linear(1,2),
        nn.Sigmoid()
    )
    self.fc = nn.Linear(2, 1)
    self._init_weights()


  def _init_weights(self): # 앞에 _(Protected Method)
    # 제이비어 초기화 함수(가중치 초기화 방법중 하나)
    nn.init.xavier_uniform_(self.layer[0].weight)
    self.layer[0].bias.data.fill_(0.01) # 편향은 간단하게 data.fill_ 로 값을 채울 수 있음

    nn.init.xavier_uniform_(self.fc.weight)
    self.fc.bias.data.fill_(0.01)


model = Net()

모델이 커지고 구조가 복잡해지면 모델의 코드도 매우 복잡해진다. 이러한 문제점을 방지하기 위해 가중치 초기화 메서드를 모듈화해 적용한다.

In [ ]:
from torch import nn


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(1, 2),
            nn.Sigmoid()
        )
        self.fc = nn.Linear(2, 1)
        self.apply(self._init_weights)
        # apply 함수는 텐서의 각 요소에 임의의 함수를 적용하고 결과와 함께 새 텐서 반환

    # module 매개변수는 초기화 메서드에서 선언한 모델의 매개변수, 적용함수에서 모델의 매개변수 전달
    def _init_weights(self, module):
        if isinstance(module, nn.Linear): # linear 변환 함수인지 확인
            nn.init.xavier_uniform_(module.weight) # 가중치 초기화
            nn.init.constant_(module.bias, 0.01) # bias 상수 초기화
        print(f"Apply : {module}")


model = Net()

Apply : Linear(in_features=1, out_features=2, bias=True)
Apply : Sigmoid()
Apply : Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): Sigmoid()
)
Apply : Linear(in_features=2, out_features=1, bias=True)
Apply : Net(
  (layer): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): Sigmoid()
  )
  (fc): Linear(in_features=2, out_features=1, bias=True)
)


하위 모듈에 재귀적으로 적용되어 self.layer 변수의 선형 변환 함수, 시그모이드 함수가 적용된 후에 시퀀셜 호출한다. 모든 계층을 호출했다면 최종적으로 네트워크까지 호출하고 종료

- 가중치 초기화함수는 p.177

## 정칙화(Regularization)

모댈이 특정 피처나 특정 패턴에 너무 많은 비중을 할당하지 않도록 손실 함수에 규제를 가해 모델의 **일반화성능(Generalization Performance)**을 향상시킨다.

정칙화를 사용하면 학습 데이터들이 갖고 있는 작은 차이점에 대해 덜 민감해져 모델의 분산 값이 낮아진다.


그리므로 정칙화는 모델이 데이터를 학습할 때 의존하는 특정의 수를 줄음으로써 모델의 추론 능력을 개선한다.

**정칙화는 모델이 비교적 복잡하고 학습에 사용되는 데이터의 수가 적을 때 활용한다.**

모델이 단순해서 매개변수가 적거나, 데이터의 수가 많거나 데이터가 잘 정제되어 있어 노이즈가 거의 없는 경우에는 사용하지 않는다.

- 정칙화 종류: L1 정칙화, L2 정칙화, 가중치감쇠, 드롭아웃 등

### L1 정칙화(Lasso Regularization)

L1 Norm 방식을 사용해 규제하는 방법(불필요한 가중치가 0에 수렴한다.)
- 특징 선택(Feature Selection) 효과를 얻을 수 있다.

L1 정칙화는 가중치 절댓값으로 계산되므로 가중치가 0에 가까워질수록 선형적인 구조를 가진다.

**사용 방법**

In [ ]:
for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)

    _lambda = 0.5
    l1_loss = sum(p.abs().sum() for p in model.parameters())

    loss = criterion(output, y) + _lambda * l1_loss



l1 정칙화는 미분할 수 없으므로 역전파를 계산하는 데 더 많은 리소스를 소모한다.

L1 정칙화는 주로 선형 모델에 적용(미분이 안 된다) L1 정칙화를 적용한 것을 라쏘 회귀(Lasso Regression)이라고 한다.

## L2 정칙화

L2 Norm 방식을 사용해 규제하는 방법(가중치를 0으로 만들지 않고 0에 가깝게 만든다.)

L2 정칙화는 가중치의 제곱으로 계산되므로 비선형적인 구조를 갖는다.


정칙화 = 손실 함수 + 정칙화값(L1 or L2)

In [ ]:
for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)

    _lambda = 0.5
    l2_loss = sum(p.pow(2.0).sum() for p in model.parameters())

    loss = criterion(output, y) + _lambda * l2_loss

과대적합을 효과적으로 방지하기 위해서는 조기 중지 또는 드롭아웃과 같은 기술과 함께 사용.

L2 정칙화는 주로 심층 신경망 모델에서 사용하며, 선형 회귀 모델에서 L2 정칙화를 적용하는 경우를 릿지회귀(Ridge Regression)라고 한다.


### 정리

||L1|L2|
|------|---|---|
|계산 방식|가중치 절대값의 합|가중치 제곱의 합|
|모델링|희소함(Sparse Solutuon)|희소하지 않음(Non-Sparse Solutuon)|
|특징선택|있음|없음|
|이상치|강함|약함|
|가중치|0이 될 수 있음|0에 가깝게 됨|
|학습|비교적 복잡한 데이터 패턴을 학습할 수 없음|비교적 복잡한 데이터 패턴을 학습할 수 있음|

## 가중치 감쇠

일반적으로 가중치 감쇠가 L2 정칙화와 동의어로 사용되지만, 가중치 감쇠는 손실 함수에 규제 항을 추가하는 기술 자체를 의미한다.

하지만 파이토치나 텐서플로 같은 딥러닝 라이브러리에서는 이 용어가 최적화 함수에 적용하는 L2 정규화 의미로 사용된다.

weight_decay 하이퍼파라미터를 설정해 구현 가능

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)

L2 정칙화를 간단하게 적용하는 방법이므로 L2 정칙화가 갖고 있는 장점과 단점을 그대로 포함

## 모멘텀

모멘텀(Momentum)은 경사 하강법 알고리즘의 변형 중 하나로, 이전에 이동했던 방향과 기울기의 크기를 고려하여 가중치를 갱신한다.

이를 위해 지수 가중 이동평균을 사용하며, 이전 기울기 값의 일부를 현재 기울기 값에 추가해 가중치 갱신.

이전 기울기 값에 의해 설정된 방향으로 더 빠르게 이동하므로, 일종의 관성(Momentum) 효과를 얻을 수 있다.


v_i = gamma*v_i-1 + 경사하강법값 (v 는 모멘텀값)

## 엘라스틱 넷
L1 정칙화와 L2 정칙화를 결합해 사용하는 방식이다. L1 정칙화는 모델이 희박한 가중치를 갖게 규제하는 반면, L2 정칙화는 큰 가중치를 갖지 않게 규제한다. 이 두 정칙화 방식을 결합함으로써 희소성과 작은 가중치의 균형을 맞춘다.

Elastic-Net = a * L1 + (1 - a) * L2

- 혼합비율 a = 1이면 L1, 0이면 L2 정칙화가 사용된다. 따라서 엘라스틱넷은 L1, L2 정칙화보다 **트레이드오프 문제를 더 유연하게 대처할 수 있다.**

- 엘라스틱 넷은 특징의 수가 샘플의 수보다 더 많을 때 유의미한 결과를 가져온다. 이로 인해 상관관계가 있는 특징을 더 잘 처리할 수 있다.

두 정칙화를 결합해 사용함으로써 각 정칙화가 가진 장점을 최대한 활용할 수 있지만, 균형적인 규제를 가하기 위해 새로운 하이퍼파라미터인 혼합 비율도 조정해야 하므로 더 많은 튜닝이 필요하게 된다.

또한 두 정칙화 모두 계산 복잡도 문제를 가지고 있으므로 더 많은 리소스 소모

## 드롭아웃(Dropout)
- 모델 평균화 효과를 얻기 위해 다른 드롭아웃 마스크를 사용해 모델을 여러번 훈련해야 해서 훈련시간이 늘어난다.
- 모든 노드를 사용해 학습하지 않으므로 데이터세트가 많지 않다면 효과를 얻기 힘들다.
- 충분한 데이터세트와 학습이 없다면 노드가 균일하게 학습될 수 없으므로 성능이 저하될 수 있다.

충분한 데이터세트와 비교적 깊은 모델에 적용.

### **적용 방식**

In [2]:
from torch import nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(10, 10)
        self.dropout = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(10, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.dropout(x)
        x = self.layer2(x)
        return x

p 는 베르누이 분포(Bernoulli Distribution)의 모수를 의미하며, 이 분포로 각 노드의 제거 여부를 확률적으로 선택한다.
- 이 과정은 순방향 메서드에서 드롭아웃을 적용할 계층 노드에 적용된다.

### **주의점**

드롭아웃은 일반적으로 배치 정규화와 동시에 사용하지 않으므로 다른 기법을 동시에 적용할 때 주의해서 적용한다.

- 드롭아웃과 배치 정규화는 서로의 정칙화 효과를 방해할 수 있다. 배치 정규화의 경우 내부 공변량 변화를 줄여 과대적합을 방지하는데, 드롭아웃은 일부 노드를 제거한다.

- 두 가지 방법을 동시에 사용하면 모델은 순방향 과정에서 다른 활성화 분포를 사용하게 된다. 이로 인해 훈련 과정에서 성능이 저하되거나 불안정해진다.

그러므로 배치 정규화를 사용하는 경우에는 드롭아웃, 배치정규화 순으로 적용한다. 또한 드롭아웃은 배치 정규화와 마찬가지로 모델이 학습할 때만 적용되며, 추론 과정에서는 일부 노드를 삭제하지 않고 모든 노드를 사용해 예측한다.
***

드롭아웃은 극단적인 비율로 모델에 적용하지 않는다면 일반적으로 성능 향상의 이점을 얻을 수 있다.

비교적 많은 특징을 사용해 학습하는 이미지 인식이나 음성 인식 모델에서 성능이 향상되는 결과를 보였다.


## 그레이디언트 클리핑(Gradient Clipping)


모델을 학습할 때 기울기가 너무 커지는 현상을 방지하는 데 사용되는 기술이다. 과대적합 모델은 특정 노드의 가중치가 너무 크다는 특징을 갖는다. 높은 가중치는 높은 분산 값을 갖게 하여 모델의 성능이 저하될 수 있다.

이러한 현상을 방지하기 위해 가중치 최대값을 규제해 최대 임계값을 초과하지 않도록 기울기를 잘라(Clipping) 설정한 임계값으로 변경한다.
(수식 190p)

그레이디언트 글리핑은 가중치 norm이 최대 임계값 r보다 높은 경우에 수행된다. 최대 임계값을 넘는 경우 기울기 벡터의 방향을 유지하면서 기울기를 잘라 규제할 수 있다.(일반적으로 L2 norm 사용해 최대 기울기 규제)
- 최대 임계값은 0.1이나 1과 같이 작은 크기의 임계값을 적용하며 학습률을 조절하는 것과 비슷한 효과를 얻을 수 있다.
- RNN 이나 LSTM 모델을 학습하는 데 주로 사용된다. 두 모델은 기울기 폭주에 취약한데, 그레이디언트 클리핑은 최대값을 억제하므로 많이 활용된다.
- 가중치 값에 대한 엄격한 제약 조건을 요구하는 상황이거나 모델이 큰 기울기에 민감한 상황에서 유용하게 활용 가능

### 적용방법

In [ ]:
for x, y in train_dataloader:
    x = x.to(device)
    y = y.to(device)

    output = model(x)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()

    # torch.nn.utils.clip_grad_norm_(parameters, max_norm, norm_type=2.0)
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
    # 전체 norm 단일 백터(grad_norm)을 반환하며, 정규화된 기울기는 반환하지 않고 매개변수 직접 수정

    optimizer.step()

역전파 수행 이후와 최적화함수 반영(optimizer.step)을 하기 전에 호출.

기울기 최대값을 규제해 비교적 큰 학습률을 사용할 수 있게 해주지만, 최대 임계값이 높으면 모델의 표현력이 떨어지며, 낮은 경우 오히려 학습이 불안정해질 수 있다.